In [1]:
#pip install polytope


In [2]:
#pip install pypoman

In [16]:
# pip install polytope
# pip install pypoman

import math
import numpy as np
from sympy.geometry import Point
import matplotlib.pyplot as plt
from pypoman import compute_polytope_halfspaces, compute_polytope_vertices
from pypoman.duality import convex_hull
import cdd
from polytope import polytope
from polytope.polytope import enumerate_integral_points, qhull, box2poly
from scipy.spatial import ConvexHull

Let's construct a reflexive 3D polytope

In [26]:
vertices = np.array([[-1, -2,  2], [ 1,  2, -2], [-3, -2,  0], [ 1,  2, -3], [ 0, -1,  2]])

In [31]:
A, b = compute_polytope_halfspaces(V)

print(A)
print(b)

[[-1.   1.   1. ]
 [ 2.  -3.5 -2. ]
 [ 5.  -5.  -2. ]
 [ 3.  -1.   0. ]
 [-1.   1.  -0. ]
 [-5.   7.   4. ]]
[1. 1. 1. 1. 1. 1.]


Pypoman provides a form $0\le -Ax+b$

In [5]:
A, b = compute_polytope_halfspaces(V)

print(A)
print(b)

print(compute_polytope_vertices(A, b))
print(convex_hull(V))

[[-1.   1.   1. ]
 [ 2.  -3.5 -2. ]
 [ 5.  -5.  -2. ]
 [ 3.  -1.   0. ]
 [-1.   1.  -0. ]
 [-5.   7.   4. ]]
[1. 1. 1. 1. 1. 1.]
[array([ 1.,  2., -3.]), array([-1., -2.,  2.]), array([ 1.,  2., -2.]), array([-0., -1.,  2.]), array([-3., -2.,  0.])]
[array([-1, -2,  2]), array([ 1,  2, -2]), array([-3, -2,  0]), array([ 1,  2, -3]), array([ 0, -1,  2])]


In [14]:
A, b = compute_polytope_halfspaces(V)

def grid_region(polyreg, res=None):
    # grid corners
    bbox = polyreg.bounding_box
    # grid resolution
    if res is None:
        density = 8
        res = [
            math.ceil(density * (b - a))
            for a, b in zip(*bbox)]
    if len(res) != polyreg.dim:
        raise ValueError((
            "`len(res)` must equal the polytope's dimension "
            "(which is {dim}), but instead `res` is:  {res}"
            ).format(dim=polyreg.dim, res=res))
    if any(n < 1 for n in res):
        raise ValueError((
            '`res` must contain `int` values >= 1, '
            'instead `res` equals:  {res}'
            ).format(res=res))
    linspaces = list()
    for a, b, n in zip(*bbox, res):
        r = np.linspace(a, b, num=n)
        linspaces.append(r)
    points = np.meshgrid(*linspaces)
    x = np.vstack(list(map(np.ravel, points)))
    x = x[:, polyreg.contains(x, abs_tol=0)]
    return (x, res)

def enumerate_stricktly_integral_points(poly):
    a, b = poly.bounding_box
    a_int = np.floor(a)
    b_int = np.ceil(b)
    intervals = list(zip(a_int.flatten(), b_int.flatten()))
    box = box2poly(intervals)
    res = [int(b - a + 1) for a, b in intervals]
    grid, _ = grid_region(box, res=res)
    inside = poly.contains(grid, abs_tol=0)
    return grid[:, inside]

integral_points = enumerate_integral_points(poly_qhull)
integral_points = integral_points.transpose()
integral_points = integral_points.astype(int)

stricktly_integral_points = enumerate_stricktly_integral_points(poly_qhull)
stricktly_integral_points = stricktly_integral_points.transpose()
stricktly_integral_points = stricktly_integral_points.astype(int)
num_stricktly_integral_points = len(stricktly_integral_points)

print("integral_points: {0}".format(integral_points.tolist()))
print("stricktly_integral_points: {0}".format(stricktly_integral_points.tolist()))
print("b: {0}".format(b))

def fitness(ip_count, distances):
    result = 0
    if ip_count > 1:
        result -= 1
    for d in distances:
        result -= abs(d-1)
    return result

print("fitness value: {0}".format(fitness(num_stricktly_integral_points, b)))

integral_points: [[-3, -2, 0], [-2, -2, 1], [-1, -2, 2], [0, -1, 2], [-1, 0, -1], [0, 0, 0], [0, 1, -2], [1, 2, -3], [1, 2, -2]]
stricktly_integral_points: [[0, -1, 2], [0, 0, 0]]
b: [1. 1. 1. 1. 1. 1.]
fitness value: -1.0


Some experimental stuff starts here:

# GA TEST

In [7]:
import math
import numpy as np
from sympy.geometry import Point
import matplotlib.pyplot as plt
from pypoman import compute_polytope_halfspaces, compute_polytope_vertices
from pypoman.duality import convex_hull
from polytope import polytope
from polytope.polytope import enumerate_integral_points, qhull, box2poly
from scipy.spatial import ConvexHull


class Population:
    def __init__(self, number_of_generations, generation_0):
        generation_0 = generation_0
        number_of_generations = number_of_generations

class Generation:
    def __init__(self, chrom_list):
        chrom_list = chrom_list


class Chromosom:
    def grid_region(polyreg, res=None):
        # grid corners
        bbox = polyreg.bounding_box
        # grid resolution
        if res is None:
            density = 8
            res = [
                math.ceil(density * (b - a))
                for a, b in zip(*bbox)]
        if len(res) != polyreg.dim:
            raise ValueError((
                "`len(res)` must equal the polytope's dimension "
                "(which is {dim}), but instead `res` is:  {res}"
                ).format(dim=polyreg.dim, res=res))
        if any(n < 1 for n in res):
            raise ValueError((
                '`res` must contain `int` values >= 1, '
                'instead `res` equals:  {res}'
                ).format(res=res))
        linspaces = list()
        for a, b, n in zip(*bbox, res):
            r = np.linspace(a, b, num=n)
            linspaces.append(r)
        points = np.meshgrid(*linspaces)
        x = np.vstack(list(map(np.ravel, points)))
        x = x[:, polyreg.contains(x, abs_tol=0)]
        return (x, res)
    
    def enumerate_stricktly_integral_points(poly):
        a, b = poly.bounding_box
        a_int = np.floor(a)
        b_int = np.ceil(b)
        intervals = list(zip(a_int.flatten(), b_int.flatten()))
        box = box2poly(intervals)
        res = [int(b - a + 1) for a, b in intervals]
        grid, _ = Chromosom.grid_region(box, res=res)
        inside = poly.contains(grid, abs_tol=0)
        return grid[:, inside]
    
    def gen_strictly_integral_points(q_hull):
        integral_points = enumerate_integral_points(q_hull)
        integral_points = integral_points.transpose()
        integral_points = integral_points.astype(int)

        stricktly_integral_points = Chromosom.enumerate_stricktly_integral_points(q_hull)
        stricktly_integral_points = stricktly_integral_points.transpose()
        stricktly_integral_points = stricktly_integral_points.astype(int)
        num_stricktly_integral_points = len(stricktly_integral_points)
        return num_stricktly_integral_points
    
    def calc_fitness(ip_count, distances):
        result = 0
        if ip_count > 1:
            result -= 1
        for d in distances:
            result -= abs(d-1)
        
        #print(result)
        return result    
    
    
    
    def __init__(self, points):
        self.gen_list = points
        self.q_hull = qhull(self.gen_list)
        self.A, self.b_half_space = compute_polytope_halfspaces(self.gen_list)

        self.num_stricktly_integral_points = Chromosom.gen_strictly_integral_points(self.q_hull)

        self.fitness = Chromosom.calc_fitness(self.num_stricktly_integral_points, self.b_half_space)
        #print(fitness)

    # def get_q_hull(self):
    #     return self.q_hull

    # def get_b_half_space(self):
    #     return self. b_half_space
    
    def get_gen_list(self):
        return self.gen_list
    
    def get_fitness(self):
        return self.fitness

    
        


In [8]:
import random
from random import randint

num_points_per_polygon = 5
num_points_per_Generation = 10

test_points = []
test_polygons = []

for j in range(0, num_points_per_Generation):
    for i in range(0,num_points_per_polygon):
        test_points.append(random.sample(range(-3, 3), 3))
    chromosome_test_points = np.array(test_points)

    chromosom_test = Chromosom(chromosome_test_points)

    test_polygons.append(chromosom_test)

#print(test_polygons)

In [9]:

test_polygons.sort(key=lambda x: x.fitness, reverse=True)

for i in test_polygons:
    print(i.get_fitness())


# a = [1,2]
# b = []
# b.append(a[0])
# b.append(a[1])
# a[1] = 5

# print(b)
    

-9.0
-61.166666666666664
-89.33333333333334
-94.44444444444446
-139.1666666666667
-145.66666666666674
-147.5
-155.16666666666669
-163.1666666666667
-164.5


In [10]:
import numpy as np
import polytope

# halfspace representation computed using `polytope`
# from the vertex representation given in the question
vertices = np.array([[0, -1], [-1, 0], [-1, 2], [1, 0], [1, -1]])
poly = polytope.qhull(vertices)
# first halfspace representation
A = np.array([
    [0, -1],
    [-1, 0],
    [-1, -1],
    [1, 1],
    [1, 0]])
b = np.array([1, 1, 1, 1, 1])
question_poly_1 = polytope.Polytope(A, b)
# second halfspace representation
A = np.array([
    [-0.70711, -0.70711],
    [-1, -0],
    [0.70711, 0.70711],
    [0, -1],
    [1, 0]])
b = np.array([0.70711, 1, 0.70711, 1, 1])

question_poly_2 = polytope.Polytope(A, b)
# check that all the above halfspace representations
# represent the same polytope
assert poly == question_poly_1, (poly, question_poly_1)
assert poly == question_poly_2, (poly, question_poly_2)

In [11]:
inner_point = np.array([0,0]) # in this case it is the origin
fac1 = polytope.quickhull.Facet([[0, -1], [-1, 0]])
fac2 = polytope.quickhull.Facet([[-1, 2], [1, 0]])
fac3 = polytope.quickhull.Facet([[1, -1], [0, -1]])

d1 = polytope.quickhull.distance(inner_point, fac1)
d2 = polytope.quickhull.distance(inner_point, fac2)
d3 = polytope.quickhull.distance(inner_point, fac3)
print(d1)
print(d2)
print(d3)

[-0.70711]
[-0.70711]
[-1.]


Let's visualize our 2D polytope. For this purpose we use the [Polytope Library](https://github.com/tulip-control/polytope)

In [12]:
# project into 2D



In [13]:

print(polytope.polytope.cheby_ball(P))

min_V = np.min(V, axis=0) #min val of each cols
min_x = min_V[0]
min_y = min_V[1]
max_V = np.max(V, axis=0) #max val of each cols
max_x = max_V[0]
max_y = max_V[1]

ax = P.plot(linestyle="solid", linewidth=1, color="lightblue", edgecolor="red")
ax.set_xlim([min_x-1, max_x+1])
ax.set_ylim([min_y-1, max_y+1])
plt.show()

NameError: name 'P' is not defined